In [45]:
.libPaths( c( .libPaths(), "/home/jovyan/libraries") )
.libPaths()[3]

[1] "/home/jovyan/libraries"

In [46]:
library(dplyr)
library(tidyr)
#install.packages('gender',lib=.libPaths()[3])
#install.packages("genderdata", repos = "http://packages.ropensci.org", type = "source",lib=.libPaths()[3])
library(gender)
library(stringr)
library(widyr)
library(readr)

In [47]:
block = read_csv("newdataCombined.csv")
myvars <- c("AuthorIdsOrder", "AuthorNamesOrder", "Year", "PaperId")
data0 <- block[myvars]
data0$Year1 <-  data0$Year-35 #25-45
data0$Year2 <-  data0$Year-45 #35-55
data0$Year3 <-  data0$Year-55 #45-65
data0$authorCount <- str_count(data0$AuthorIdsOrder, '; ')+1

Parsed with column specification:
cols(
  Tag = col_character(),
  PaperId = col_double(),
  AuthorIdsOrder = col_character(),
  AuthorNamesOrder = col_character(),
  FoSNames = col_character(),
  Year = col_double(),
  DocType = col_character(),
  Journal = col_character(),
  Publisher = col_character(),
  Doi = col_character(),
  Title = col_character(),
  EstimatedCitation = col_double(),
  IndexedAbstract = col_character()
)


In [48]:
AuthorTable <- data0 %>% separate(AuthorNamesOrder, into = sprintf('%s.%s', rep('Author',100), rep(1:100)), sep = "; ") #max author has exceeded 90
AuthorTable <- AuthorTable %>% gather(authorOrder, name, into = sprintf('%s.%s', rep('Author',100), rep(1:100)))
AuthorList <- data.frame(lapply(AuthorTable, trimws), stringsAsFactors = FALSE)
nameTable <- AuthorList %>% separate(name, into = sprintf('%s.%s', rep('namePart',4), rep(1:4)), sep = " ")

Warning message:
“Expected 100 pieces. Missing pieces filled with `NA` in 3001 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”Warning message:
“Expected 4 pieces. Additional pieces discarded in 113 rows [101, 333, 1127, 1145, 1165, 1228, 1307, 1611, 1771, 1844, 2138, 2199, 2402, 2535, 2573, 2600, 2666, 2720, 2772, 2898, ...].”Warning message:
“Expected 4 pieces. Missing pieces filled with `NA` in 13200 rows [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, ...].”

In [49]:
newvars <- c("PaperId","namePart.1", "namePart.2", "namePart.3", "namePart.4", "Year2", "authorOrder", "authorCount")
nameParts <- nameTable[newvars]
nameParts$namePart.1 <- gsub("\"", "", nameParts$namePart.1)
nameParts$namePart.2 <- gsub("\"", "", nameParts$namePart.2)
nameParts$namePart.3 <- gsub("\"", "", nameParts$namePart.3)
nameParts$namePart.4 <- gsub("\"", "", nameParts$namePart.4)
nameParts$min_years <- as.numeric(nameParts$Year2)-20
nameParts$max_years <- as.numeric(nameParts$Year2)+20
results <- gender_df(nameParts, name_col = "namePart.1", year_col = c("min_years", "max_years"), method = "ssa")
output <- nameParts %>% left_join(results, by = c("namePart.1" = "name", "min_years" = "year_min"))
results <- gender_df(nameParts, name_col = "namePart.2", year_col = c("min_years", "max_years"), method = "ssa")
output <- output %>% left_join(results, by = c("namePart.2" = "name", "min_years" = "year_min"))
results <- gender_df(nameParts, name_col = "namePart.3", year_col = c("min_years", "max_years"), method = "ssa")
output <- output %>% left_join(results, by = c("namePart.3" = "name", "min_years" = "year_min"))
output <- within(output, #if namepart3 is the last name, set it NA 
   temp2 <- ifelse(is.na(namePart.4),NA,proportion_female)
)
output <- within(output, #if namepart2 is the last name, set it NA 
   temp1 <- ifelse(is.na(namePart.3),NA,proportion_female.y)
)
output <- within(output, #if namepart1 is the last name, set it NA 
   temp0 <- ifelse(is.na(namePart.2),NA,proportion_female.x)
)
output <- within(output, # namepart2 merged with namepart3
   femaleProb0 <- ifelse(is.na(temp1),temp2,temp1)
)
output <- within(output, # namepart1 merged with namepart2
   femaleProb <- ifelse(is.na(temp0),temp1,temp0)
)
output <- within(output, # binarize the probabilities, with 1 representing female and 0 male
   genderLabel <- ifelse(femaleProb>0.5,1,0)
)
output %>% 
  group_by(genderLabel) %>%
  summarise(no_rows = length(genderLabel))

genderLabel,no_rows
0,6256
1,3210
NA,290634


In [102]:
newvars <- c("PaperId", "Year2", "authorOrder", "genderLabel", "authorCount")
merged <- output[newvars]
merged <- spread(merged, authorOrder, genderLabel)
merged$detectCount <- 100-rowSums(is.na(merged))
merged$femaleCount <- rowSums(merged[sapply(merged, is.numeric)], na.rm = TRUE) - merged$detectCount
merged$femaleProp <- merged$femaleCount/merged$detectCount
merged$femaleProp[is.infinite(merged$femaleProp) | is.nan(merged$femaleProp) ] <- NA
merged$authorCount <- as.numeric(merged$authorCount)
merged$unknownCount <- merged$authorCount-merged$detectCount
merged$lastColName <- paste("Author", merged$authorCount, sep=".") 
merged$last <- as.numeric(merged[cbind(1:nrow(merged),match(merged$lastColName, colnames(merged)))])
merged$PaperId <- as.numeric(merged$PaperId)
merged <- rename(merged, "1st" = "Author.1", "2nd" = "Author.2", "3rd" = "Author.3", "4th" = "Author.4", "5th" = "Author.5")
outputVars <- c("PaperId", "authorCount", "unknownCount", "femaleCount", "femaleProp", "1st", "2nd", "3rd", "4th", "5th", "last")
mergedSelect <-  merged[outputVars]
#write.csv(merged, file = "output.csv")

In [103]:
block0 <- block %>% left_join(merged, by = c("PaperId" = "PaperId"))
outputVars <- c("Tag", "PaperId", "AuthorIdsOrder", "AuthorNamesOrder", "authorCount", "unknownCount", "femaleCount", "femaleProp", "1st", "2nd", "3rd", "4th", "5th", "last",
                "FoSNames", "Year", "DocType", "Journal", "Publisher", "Doi", "Title", "EstimatedCitation", "IndexedAbstract")
data <- block0[outputVars]
write_csv(data, "OpenSci3.csv")
str(data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	3001 obs. of  23 variables:
 $ Tag              : chr  "OpenScience" "OpenScience" "OpenScience" "OpenScience" ...
 $ PaperId          : num  2.63e+09 1.48e+09 2.32e+09 2.32e+09 1.97e+09 ...
 $ AuthorIdsOrder   : chr  "1918640562" "1971373124" "115656586" "1908890355; 2136934135; 2145472093; 2045166491; 2006167755; 2303251601; 2127190699; 2128270114; 2081974026; 2154796455" ...
 $ AuthorNamesOrder : chr  "Heidi Laine" "Jonas Kubilius" "Elliott M. Antman" "Elise Ruark; Nazneen Rahman; Anthony Renwick; Matthew Clarke; Katie Snape; Emma Ramsay; Anna Elliott; Sandra Ha"| __truncated__ ...
 $ authorCount      : num  1 1 1 10 1 4 3 7 2 3 ...
 $ unknownCount     : num  0 0 0 1 0 1 0 0 0 0 ...
 $ femaleCount      : num  1 0 0 7 0 1 1 2 1 3 ...
 $ femaleProp       : num  1 0 0 0.778 0 ...
 $ 1st              : num  1 0 0 1 0 0 1 0 0 1 ...
 $ 2nd              : num  NA NA NA NA NA NA 0 0 1 1 ...
 $ 3rd              : num  NA NA NA 0 NA 0 0 1 NA 1 ...
 $ 

In [107]:
data$allmale <- as.numeric(data$"1st"==0) * as.numeric(data$last==0) * as.numeric(data$authorCount>1)
data$allfemale <- as.numeric(data$"1st"==1) * as.numeric(data$last==1) * as.numeric(data$authorCount>1)
data$mixed <- as.numeric(data$"1st"==1) + as.numeric(data$last==1)
data$mixed <- as.numeric(data$mixed==1)
data$allUnknown <- as.numeric(is.na(data$"1st")) * as.numeric(is.na(data$last)) * as.numeric(data$authorCount>1)
data$UnknownMale <- as.numeric(is.na(data$"1st")) * as.numeric(data$last==0)
data$UnknownFemale <- as.numeric(is.na(data$"1st")) * as.numeric(data$last==1)
data$maleUnknown <- as.numeric(is.na(data$last)) * as.numeric(data$"1st"==0)
data$femaleUnknown <- as.numeric(is.na(data$last)) * as.numeric(data$"1st"==1)
data$singleMale <- as.numeric(data$"1st"==0) * as.numeric(data$authorCount==1)
data$singleFemale <- as.numeric(data$"1st"==1) * as.numeric(data$authorCount==1)
data$singleUnknown <- as.numeric(is.na(data$"1st")) * as.numeric(data$authorCount==1)
outputVars <- c("Tag","allmale","allfemale","mixed","allUnknown","UnknownMale","UnknownFemale","maleUnknown","femaleUnknown","singleMale","singleFemale","singleUnknown")
dataOpen <- data[outputVars]
dataOpen <- dataOpen[dataOpen$Tag=="OpenScience",]
pieOpen <- dplyr::bind_rows(colSums(dataOpen[,-1], na.rm = TRUE))
dataRep <- data[outputVars]
dataRep <- dataRep[dataRep$Tag=="Reproducibility",]
pieRep <- dplyr::bind_rows(colSums(dataRep[,-1], na.rm = TRUE))
pie <- dplyr::bind_rows(list(OpenScience=pieOpen, Reproducibility=pieRep), .id = 'Tag')
pie$sum <- rowSums(pie[-1])
write_csv(pie, "Pie.csv")